In [1]:
import lilac as ll


/Users/dsmilkov/code/lilac/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ds = ll.get_dataset('local', 'open-hermes')


In [6]:
def dup(x):
  return x['input'] + x['instruction']


ds.map(dup, output_column='input_instruction', overwrite=True)

# counts = ds.select_groups('input_instruction').counts


Scheduling task "1463336797044730991d161cb78eac2d": "[local/open-hermes][shard 0/1] map "dup" to "input_instruction"".


[local/open-hermes][shard 0/1] map "dup" to "input_instruction": 100%|██████████| 242831/242831 [00:03<00:00, 71620.01it/s]


Wrote map output to ./data/datasets/local/open-hermes/./data/datasets/local/open-hermes/input_instruction-00000-of-00001.parquet


Task finished "1463336797044730991d161cb78eac2d": "[local/open-hermes][shard 0/1] map "dup" to "input_instruction"" in 3s.


In [10]:
ds.compute_signal(ll.NearDuplicateSignal(threshold=0.7), 'input_instruction')


Fingerprinting...: 242831it [00:23, 10159.66it/s]":0.7,"signal_name":"near_dup"}) over ('input_instruction',):   0%|          | 0/242831 [00:00<?, ?it/s]
2023-11-28 08:46:38,429 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2023-11-28 08:46:40,034 - distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
2023-11-28 08:46:41,687 - distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
2023-11-28 08:46:44,898 - distributed.utils_perf - WARNING - full garbage collections took 18% CPU time recently (threshold: 10%)
Clustering...: 100%|██████████| 25/25 [00:00<00:00, 27.55it/s]
2023-11-28 08:46:46,479 - distributed.utils_perf - WARNING - full garbage collections took 17% CPU time recently (threshold: 10%)
Computing signal  NearDuplicateSignal({"threshold":0.7,"signal_name":"near_dup"}) over ('input_instruction',):   0%|          | 1/242

Wrote signal output to ./data/datasets/local/open-hermes/input_instruction/near_dup%28threshold%3D0.7%29


In [12]:
ds.manifest().data_schema


{
  "fields": {
    "input": {
      "dtype": {
        "type": "string"
      }
    },
    "output": {
      "dtype": {
        "type": "string"
      }
    },
    "instruction": {
      "dtype": {
        "type": "string"
      }
    },
    "__hfsplit__": {
      "dtype": {
        "type": "string"
      }
    },
    "input_instruction": {
      "fields": {
        "near_dup": {
          "fields": {
            "cluster_id": {
              "dtype": {
                "type": "uint32"
              },
              "categorical": true
            }
          },
          "signal": {
            "threshold": 0.85,
            "signal_name": "near_dup"
          }
        },
        "near_dup(threshold=0.7)": {
          "fields": {
            "cluster_id": {
              "dtype": {
                "type": "uint32"
              },
              "categorical": true
            }
          },
          "signal": {
            "threshold": 0.7,
            "signal_name": "near_dup"
   

In [8]:
counts = ds.select_groups('input_instruction').counts
num_duplicates = 0
for _, c in counts:
  num_duplicates += c - 1

print('Number of duplicates: ' + str(num_duplicates))


Number of duplicates: 9152


In [9]:
counts = ds.select_groups('input_instruction.near_dup.cluster_id').counts
num_duplicates = 0
for _, c in counts:
  num_duplicates += c - 1

print('Number of near-duplicates: ' + str(num_duplicates))


Number of near-duplicates: 9545


In [13]:
counts = ds.select_groups('input_instruction."near_dup(threshold=0.7)".cluster_id').counts
num_duplicates = 0
for _, c in counts:
  num_duplicates += c - 1

print('Number of near-duplicates: ' + str(num_duplicates))


Number of near-duplicates: 11250
